In [1]:
import getopt
import sys
import gc
import os
import random
sys.path.append('/home/atac2rna/program/atac2rna/Model/butterfly/Butterfly')
sys.path.append('/home/atac2rna/program/atac2rna/Model/butterfly/experiment/RNA_ADT')
from data_processing import RNA_data_preprocessing, ATAC_data_preprocessing
import scanpy as sc
import anndata as ad

sc_data = sc.read_h5ad('/home/atac2rna/data/atac2rna/data/openproblems_neurips2021/GSE194122_openproblems_neurips2021_multiome_BMMC_count.h5ad')
RNA_data_test = sc_data[:, 0:13431]
ATAC_data_test = sc_data[:, 13431:]
del sc_data
gc.collect()

RNA_data_train = sc.read_h5ad('/home/atac2rna/data/atac2rna/data/openproblems_neurips2021/bmmc_RNA_cnt.h5ad')
ADT_data_train = sc.read_h5ad('/home/atac2rna/data/atac2rna/data/openproblems_neurips2021/bmmc_ADT_CLRed.h5ad')

In [ ]:
rna_atac_gene = RNA_data_test.var.index
rna_adt_gene = RNA_data_train.var.index
gene_list_rna_atac = []
gene_list_rna_adt = []
for i in range(len(rna_atac_gene)):
    for j in range(len(rna_adt_gene)):
        if rna_adt_gene[j] == rna_atac_gene[i]:
            gene_list_rna_atac.append(i)
            gene_list_rna_adt.append(j)
            break

In [ ]:
import scipy.sparse as sp

In [ ]:
big_RNA = ad.AnnData(X = sp.vstack((RNA_data_train.X[:, gene_list_rna_adt], RNA_data_test.X[:, gene_list_rna_atac])))

In [ ]:
big_RNA.var = RNA_data_test.var.iloc[gene_list_rna_atac, :]

In [ ]:
big_RNA.var

In [ ]:
file_path = '/home/atac2rna/data/atac2rna/model_output/22_10_12/Cite_test/A2R2A/A2R_batch_4'

In [8]:
############################################################
# Part 1 data processing
big_RNA = RNA_data_preprocessing(
    big_RNA,
    normalize_total=True,
    log1p=True,
    use_hvg=True,
    n_top_genes=3000,
    save_data=False,
    file_path=file_path,
    logging_path=file_path
    )

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [9]:
ATAC_data_test = ATAC_data_preprocessing(
    ATAC_data_test,
    binary_data=True,
    filter_features=True,
    fpeaks=0.005,
    tfidf=True,
    normalize=True,
    save_data=False,
    file_path=file_path,
    logging_path=file_path
)[0]

1


2
3


In [ ]:
RNA_data_train = ad.AnnData(X=big_RNA.X[0:90261, :], obs=RNA_data_train.obs, var=big_RNA.var)

In [ ]:
RNA_data_test = ad.AnnData(X=big_RNA.X[90261:, :], obs=RNA_data_test.obs, var=big_RNA.var)

In [ ]:
RNA_data_test

In [ ]:
number = 4

In [ ]:
############################################################
# Part 2 split datasets
from split_datasets import *
id_list = bmmc_batch_split_dataset(RNA_data_test, ATAC_data_test)
train_id, validation_id, test_id = id_list[int(number) - 1]
train_id_r = train_id.copy()
train_id_a = train_id.copy()
validation_id_r = validation_id.copy()
validation_id_a = validation_id.copy()
test_id_r = test_id.copy()
test_id_a = test_id.copy()

In [ ]:
from scvi_colab import install
import pandas as pd
install()
import scvi
import sys
from calculate_cluster import *
import scipy.sparse as sp
from split_datasets import *
import gc

sc_data = sc.read_h5ad('/home/atac2rna/data/atac2rna/data/openproblems_neurips2021/GSE194122_openproblems_neurips2021_multiome_BMMC_count.h5ad')
ATAC_data = sc_data[:, 13431:]
RNA_data = sc_data[:, 0:13431]
del sc_data
gc.collect()

adata = ad.AnnData(sp.hstack((RNA_data.X, ATAC_data.X)))
adata.X = adata.X.tocsr()
adata.obs = RNA_data.obs

m = len(RNA_data.var.index)
n = len(ATAC_data.var.index)
adata.var.index = pd.Series([RNA_data.var.index[i] if i<m else ATAC_data.var.index[i-m] for i in range(m+n)], dtype='object')
adata.var['modality'] = pd.Series(['Gene Expression' if i<m else 'Peaks' for i in range(m+n)], dtype='object').values

adata.var_names_make_unique()

adata_mvi = scvi.data.organize_multiome_anndatas(adata)

adata_mvi = adata_mvi[:, adata_mvi.var["modality"].argsort()].copy()

sc.pp.filter_genes(adata_mvi, min_cells=int(adata_mvi.shape[0] * 0.01))


id_list = bmmc_batch_split_dataset(RNA_data, ATAC_data, seed = 19193)


train_id, validation_id, test_id = id_list[int(number) - 1]

train_adata = idx2adata_multiVI(adata_mvi, train_id, validation_id, test_id)[0]

scvi.model.MULTIVI.setup_anndata(train_adata, batch_key='modality')

mvi = scvi.model.MULTIVI(
    train_adata,
    n_genes=(adata_mvi.var['modality']=='Gene Expression').sum(),
    n_regions=(adata_mvi.var['modality']=='Peaks').sum(),
)

mvi.train()

train_adata.obsm["MultiVI_latent"] = mvi.get_latent_representation()
leiden_adata = ad.AnnData(train_adata.obsm["MultiVI_latent"])
sc.pp.neighbors(leiden_adata)
sc.tl.leiden(leiden_adata, resolution=3)

del train_adata, adata_mvi
gc.collect()

In [ ]:
import random
copy_count = 3
random.seed(19193)
cell_type = leiden_adata.obs.leiden
for i in range(len(cell_type.cat.categories)):
    cell_type_name = cell_type.cat.categories[i]
    idx_temp = list(cell_type[cell_type == cell_type_name].index.astype(int))
    for j in range(copy_count - 1):
        random.shuffle(idx_temp)
        for each in idx_temp:
            train_id_r.append(train_id[each])
        random.shuffle(idx_temp)
        for each in idx_temp:
            train_id_a.append(train_id[each])

In [16]:
len(train_id_r)

95397

In [17]:
chrom_list = []
last_one = ''
for i in range(len(ATAC_data_test.var.gene_id.index)):
    temp = ATAC_data_test.var.gene_id.index[i].split('-')[0]
    if temp[0 : 3] == 'chr':
        if not temp == last_one:
            chrom_list.append(1)
            last_one = temp
        else:
            chrom_list[-1] += 1
    else:
        chrom_list[-1] += 1

In [18]:
from train_model import Model
import torch
import torch.nn as nn

RNA_input_dim = len([i for i in RNA_data_test.var['highly_variable'] if i])
ATAC_input_dim = ATAC_data_test.X.shape[1]

R_kl_div = 1 / RNA_input_dim * 20
A_kl_div = 1 / ATAC_input_dim * 20
kl_div = R_kl_div + A_kl_div

############################################################
# Part 4 construct model
model = Model(
    R_encoder_nlayer = 2, 
    A_encoder_nlayer = 2,
    R_decoder_nlayer = 2, 
    A_decoder_nlayer = 2,
    R_encoder_dim_list = [RNA_input_dim, 256, 128],
    A_encoder_dim_list = [ATAC_input_dim, 32 * len(chrom_list), 128],
    R_decoder_dim_list = [128, 256, RNA_input_dim],
    A_decoder_dim_list = [128, 32 * len(chrom_list), ATAC_input_dim],
    R_encoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    A_encoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    R_decoder_act_list = [nn.LeakyReLU(), nn.LeakyReLU()],
    A_decoder_act_list = [nn.LeakyReLU(), nn.Sigmoid()],
    translator_embed_dim = 128, 
    translator_input_dim_r = 128,
    translator_input_dim_a = 128,
    translator_embed_act_list = [nn.LeakyReLU(), nn.LeakyReLU(), nn.LeakyReLU()],
    discriminator_nlayer = 1,
    discriminator_dim_list_R = [128],
    discriminator_dim_list_A = [128],
    discriminator_act_list = [nn.Sigmoid()],
    dropout_rate = 0.1,
    R_noise_rate = 0.5,
    A_noise_rate = 0.3,
    chrom_list = chrom_list,
    logging_path = file_path,
    RNA_data = RNA_data,
    ATAC_data = ATAC_data
)

In [ ]:
model.train(
    R_encoder_lr = 0.001,
    A_encoder_lr = 0.001,
    R_decoder_lr = 0.001,
    A_decoder_lr = 0.001,
    R_translator_lr = 0.001,
    A_translator_lr = 0.001,
    translator_lr = 0.001,
    discriminator_lr = 0.005,
    R2R_pretrain_epoch = 100,
    A2A_pretrain_epoch = 100,
    lock_encoder_and_decoder = False,
    translator_epoch = 200,
    patience = 50,
    batch_size = 64,
    r_loss = nn.MSELoss(size_average=True),
    a_loss = nn.BCELoss(size_average=True),
    d_loss = nn.BCELoss(size_average=True),
    loss_weight = [1, 2, 1, R_kl_div, A_kl_div, kl_div],
    train_id_r = train_id_r,
    train_id_a = train_id_a,
    validation_id_r = validation_id_r, 
    validation_id_a = validation_id_a, 
    output_path = file_path,
    seed = 19193,
    kl_mean = True,
    R_pretrain_kl_warmup = 50,
    A_pretrain_kl_warmup = 50,
    translation_kl_warmup = 50,
    load_model = None,
    logging_path = file_path
)

RNA pretrain:   1%|▏                   | 1/100 [03:45<6:11:26, 225.12s/it, train=0.1062, val=0.1206]

In [21]:
model.is_train_finished

True